# 채용 사이트  스크래핑 

In [1]:
from selenium import webdriver
import time

In [2]:
browser = webdriver.Chrome('./chromedriver.exe')

In [3]:
browser.get('https://www.incruit.com/')

In [4]:
browser.get('https://job.incruit.com/jobdb_list/searchjob.asp?occ2=632')

### bs 사용

In [5]:
from bs4 import BeautifulSoup

In [6]:
html = BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
tags = html.select('div.n_job_list_table_a > table > tbody > tr')
type(tags), len(tags)

(bs4.element.ResultSet, 60)

In [8]:
tag = tags[0]

In [9]:
# Company Name
tag.select('div.companys > div > span > a')[0]['title']

'남서울대학교'

In [10]:
# Due Date
tag.select('div.ddays > p')[1].text.strip()

'~07.06 (화)'

In [11]:
uri = 'https://job.incruit.com/jobdb_list/searchjob.asp?occ2=632&page='

results = [] # or list()

for page in range(1,6) :
    target = uri + str(page)
    browser.get(target)
    html = browser.page_source
    
    soup = BeautifulSoup(html, 'html.parser')
    tags = soup.select('div.n_job_list_table_a > table > tbody > tr')
    
    for tag in tags:
        name = tag.select('div.companys > div > span > a')[0]['title']
        if len(tag.select('div.ddays > p')) == 2 :
            date = tag.select('div.ddays > p')[1].text.strip()
        else :
            date = tag.select('div.ddays > p')[0].text.strip()
        results.append([name, date])

len(results)

300

In [12]:
import pandas as pd

In [13]:

df = pd.DataFrame(results, columns=['Company_Name','Due_Data'])
df

,Company_Name,Due_Data
0,남서울대학교,~07.06 (화)
1,대구경북첨단의료산업진흥재단,~07.12 (월)
2,재단법인 서울산업진흥원,~07.12 (월)
3,(주)더존비즈온,~07.11 (일)
4,한국산지보전협회,~06.30 (수)
...,...,...
295,(주)엔씨소프트,~07.30 (금)
296,파워에이치알,채용시
297,파워에이치알,채용시
298,(주)엔씨소프트,~07.15 (목)


In [14]:

df.to_excel('./saves/incruit_scraping.xls',index=False)